In [ ]:
import pandas as pd
import glob

#先不讀取圖片，真的要訓練才讀
#glob是顯示圖片路徑
catfn = glob.glob("dataset/train/cat.*")
catans = [0] *len(catfn)
dogfn = glob.glob("dataset/train/dog.*")
dogans = [1] *len(dogfn)

#"target":catans +dogans =矩陣相加
df = pd.DataFrame({
    "path":catfn +dogfn,
    "target":catans +dogans
})
df

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
p = "dataset/train/dog.3014.jpg"
img = load_img(p)
print(img.size)
img

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
#include_top  需不需要模型中的MLP  設成False 
# input_shape 當初訓練是(224,224,,3)的圖形，建議設成跟當初差不多的大小
#cnn.layers= list
cnn = VGG16(include_top=False,input_shape=(224,224,3))
cnn.summary()
#Total params: 14,714,688
#Trainable params: 14,714,688,Trainable照著負斜率去調整
#Non-trainable params: 0  ,Non-trainable1不照著負斜率去調整
cnn.layers

In [ ]:
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.models import Sequential
model = Sequential()
for i in cnn.layers:
  #trainable一定要設,因為如果都照著負斜率去調整，會被一開始的CNN拖累，所以設了此條件，之後Dense的參數就不會受到VGG16模型影響 compile前要設定好
  i.trainable = False
  model.add(i)
  
#資料很多就可以有很多Dense參數，資料少就用較少的參數
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(2, activation="softmax"))
model.summary()
#Trainable params: 14,714,688,Trainable  =  照著負斜率去調整
#Non-trainable params: 0  ,Non-trainable1  =  不照著負斜率去調整
#Sequential()只能一層疊一層，之後會教如何平行建立


In [ ]:
#Flatten()(cnn.output)的效果等於
# def test():
    #return int
#test()(3.14) =3
#model的效果等同於sequential，equential只能一層疊一層，model更有彈性，可以一次傳入兩個圖片，做平行的合併
#如果不是  tensorflow compile做的，就要Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
for l in cnn.layers:
    l.trainable = False
x = BatchNormalization()(cnn.output)
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.25)(x)
out = Dense(2, activation="softmax")(x)
model = Model(inputs=cnn.input, outputs=out)
model.summary()

In [ ]:
#自動


from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])



In [ ]:
# 好習慣: 全部轉成numpy array
import numpy as np
from sklearn.model_selection import train_test_split
x, y = np.array(df["path"]), np.array(df["target"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
x_train

In [ ]:
#這是預處理?用cafe版本的預處理  每個值都-127.5
#之所以要遇處理圖片，是因為框架有假設一個範圍(此案例是用cafe方式讓值在一定範圍)

#BATCH=10 10筆一次調整
# (Tensor). tensorflow運算中最基本的單位，通常是高維度的矩陣
#preprocess_input  =載入VGG比較喜歡的預處理方式對資料作處理
from tensorflow.keras.applications.vgg16 import preprocess_input

def dataflow(x, y, BATCH=10):
    idx = np.random.randint(0, x.shape[0], size=BATCH)
    img_raw, img_process = [], []
    for p in x[idx]:
        img = load_img(p, target_size=(224, 224))
        img_np = np.array(img)
        img_raw.append(img_np)
        img_pre = preprocess_input(img_np)
        img_process.append(img_pre)
    return (np.array(img_raw), 
            np.array(img_process), 
            np.array(y[idx]))
    
data = dataflow(x_train, y_train)
print(data[0].shape, data[1].shape, data[2].shape)
# return value: (原圖, 預處理過圖, 答案)



In [ ]:

for i in range(10):
    print("-" * 15, i, "-" * 15)
    raw, img, ans = dataflow(x_train, y_train)
    train_result = model.train_on_batch(img, ans)
    print("Train:", train_result)
    raw, img, ans = dataflow(x_test, y_test)
    test_result = model.test_on_batch(img, ans)
    print("Validate:", test_result)

#自寫訓練函式
#因為cafe放大了輸出值127倍 ，所以出來的值會非常右邊及非常左邊，趨近於0  or 1

In [ ]:


# 正常機率: [很小很小, 很大很大]
# 不正常機率: [1, 0]
model.predict(img)



In [ ]:
raw,img,ans = dataflow(x_test, y_test,100)
model.evaluate(img,ans)

In [ ]:
#Model(非SEquential):沒有predict_classes
#argmax = 找最大值索引
#axis=1 =找每一行(筆)的最大值
model.predict(img).argmax(axis=1)
pre

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

names = ["Cat", "Dog"]
idx = np.nonzero(pre != ans)[0]
false_pre_img = raw[idx]
false_pre_label = ans[idx]
false_pre_predict = pre[idx]

width = 10
height = len(idx) // width + 1
plt.figure(figsize=(14, 20))
for i, img in enumerate(false_pre_img):
    plt.subplot(height, width, i+1)
    t = "[P]:{}\n[O]:{}".format(names[false_pre_predict[i]],
                                names[false_pre_label[i]])
    plt.title(t)
    plt.axis("off")
    plt.imshow(img)